# Data Masters Case: Modelo de Classificação

Felipe Viacava -- São Paulo, ago/2023

O presente documento consiste na avaliação de modelos de classificação como parte da solução do Case "Data Masters - Cientista de Dados" do Santander Brasil. O objetivo é treinar um modelo de classificação de clientes possivelmente insatisfeitos com o banco para selecionar o público alvo de uma campanha de retenção.

Além das premissas estabelecidas na etapa de exploração dos dados, assumimos que:
- Buscamos um modelo de alto poder preditivo e não um modelo para inferência;
- Para cada cliente marcado como insatisfeito (_predicted positive_) pelo modelo, temos um custo de R$10,00;
- Destes, para os verdadeiramente insatisfeitos (_true positives_), temos um retorno de R$100,00;
- O melhor modelo é aquele que maximiza o retorno sobre a campanha de retenção, ou seja, a soma de:
    - _False Positives_, que geram custo R$10,00 por observação;
    - _True Positives_, que geram lucro de R$90,00 por observação;
    - _False Negatives_, que não geram lucro nem custo por observação;
    - _True Negatives_, que não geram lucro nem custo por observação.

As premissas acima surgem da interpretação do enunciado do problema. Outras premissas que nortearam o desenvolvimento dos modelos são:
- O código deve ser limpo, conciso e reprodutível;
- A performance dos modelos deve ser comparada sobre um conjunto de dados nunca antes visto, prevenindo _data leakage_. Este conjunto foi separado antes da análise exploratória, simulando o que seria o acompanhamento da performance de um modelo após sua implementação;
- Uso de transformadores em _pipelines_ para qualquer tipo de processamento nos dados, garantindo fácil implementação e reprodutibilidade;
- Todos os modelos testados devem obrigatoriamente passar pelos mesmos passos, sendo estes:
    - Divisão dos dados entre treino e validação, estratificado por TARGET;
    - _GridSearchCV_ para hiperparametrização maximizando a _AUC_ (conjunto de treino);
    - Retreino após a escolha dos melhores hiperparâmetros (conjunto de treino);
    - Ajuste do corte de classificação maximizando a função $\frac{90tp - 10fp}{n}$ (conjunto de validação);
    - Retreino após a escolha do melhor corte de classificação e melhores hiperparâmetros (conjunto de treino + validação);
    - Avaliação e comparação no contexto de negócios (conjunto de testes).

## Bibliotecas

In [1]:
# --- Data Exploration and Viz --- #
import pandas as pd

# --- Models --- #
import pickle
from resources.train_evaluate import TrainEvaluate

## Leitura dos dados

In [2]:
test = pd.read_csv("data/test.csv")

## Modelagem

### Random Forest

In [3]:
#| label: rf-pipeline
#| echo: true
#| code-fold: true
#| code-summary: "Mostrar/esconder código"

with open("models/rf.pkl", "rb") as f:
    rf = pickle.load(f)

rf.best_model_

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('prep',
                                  Pipeline(steps=[('DropConstantColumns',
                                                   DropConstantColumns(also=['ID'])),
                                                  ('DropDuplicateColumns',
                                                   DropDuplicateColumns()),
                                                  ('NoneZeroCountSaldo',
                                                   AddNonZeroCount(prefix='saldo')),
                                                  ('SumSaldo',
                                                   CustomSum(prefix='saldo')),
                                                  ('NoneZeroCountImp',
                                                   AddNonZeroCount(prefix='imp')),
                                                  ('SumImp',
                                                   CustomSum(prefix='...
                                                   CustomEncoder(colname='var36')),
                                                  ('CustomEncoderVar21',
                                                   CustomEncoder(colname='var21'))])),
                                 ('NoneCountVar3', AddNoneCount(prefix='var3')),
                                 ('drop_almost',
                                  DropConstantColumns(ignore_prefix='ind',
                                                      thresh=0.99)),
                                 ('nan', SimpleImputer(strategy='median'))])),
                ('classifier',
                 RandomForestClassifier(class_weight='balanced', max_depth=8,
                                        max_features=64, n_estimators=500,
                                        random_state=42))])

In [4]:
#| label: rf-params
#| echo: true
#| code-fold: true
#| code-summary: "Mostrar/esconder código"
rf.param_grid

{'classifier__max_depth': [4, 8, 16, 32],
 'classifier__max_features': [8, 16, 32, 64]}

### Histogram Gradient Boosting

In [5]:
#| label: hgb-pipeline
#| echo: true
#| code-fold: true
#| code-summary: "Mostrar/esconder código"
with open("models/hgb.pkl", "rb") as f:
    hgb = pickle.load(f)

hgb.best_model_

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('DropConstantColumns',
                                  DropConstantColumns(also=['ID'])),
                                 ('DropDuplicateColumns',
                                  DropDuplicateColumns()),
                                 ('NoneZeroCountSaldo',
                                  AddNonZeroCount(prefix='saldo')),
                                 ('SumSaldo', CustomSum(prefix='saldo')),
                                 ('NoneZeroCountImp',
                                  AddNonZeroCount(prefix='imp')),
                                 ('SumImp', CustomSum(prefix='imp')),
                                 ('ImputeNanDelta'...
                                  CustomImputer(prefix='var3',
                                                to_replace=-999999)),
                                 ('CustomEncoderVar36',
                                  CustomEncoder(colname='var36')),
                                 ('CustomEncoderVar21',
                                  CustomEncoder(colname='var21'))])),
                ('classifier',
                 HistGradientBoostingClassifier(categorical_features=['var36',
                                                                      'var21'],
                                                class_weight='balanced',
                                                l2_regularization=10,
                                                learning_rate=0.03, max_depth=4,
                                                max_iter=150,
                                                random_state=42))])

In [6]:
#| label: hgb-params
#| echo: true
#| code-fold: true
#| code-summary: "Mostrar/esconder código"
hgb.param_grid

{'classifier__learning_rate': [0.01, 0.03, 0.1],
 'classifier__max_iter': [150, 200, 250],
 'classifier__max_depth': [3, 4, 5, 6, 8],
 'classifier__l2_regularization': [0.3, 1, 3, 10]}

## Comparação

In [7]:
#| label: compare-business
#| echo: true
#| code-fold: true
#| code-summary: "Mostrar/esconder código"
rf = rf.evaluate(test)
hgb = hgb.evaluate(test)

business_results = pd.DataFrame(
    data = [
        rf.business_metrics,
        hgb.business_metrics
    ],
    index = [
        "Random Forest",
        "Histogram-based Gradient Boosting"
    ]
)

business_results.transpose().applymap(lambda x: f"R${x:,.2f}")

,Random Forest,Histogram-based Gradient Boosting
Profit (Total),"R$17,520.00","R$19,060.00"
Profit (per Customer),R$0.92,R$1.00
True Positive Profit (Total),"R$36,720.00","R$39,870.00"
True Positive Profit (per Customer),R$1.93,R$2.10
False Positive Loss (Total),"R$19,200.00","R$20,810.00"
False Positive Loss (per Customer),R$1.01,R$1.09
False Negative Potential Profit Loss (Total),"R$30,960.00","R$27,810.00"
False Negative Potential Profit Loss (per Customer),R$1.63,R$1.46
True Negative Loss Prevention (Total),"R$163,330.00","R$161,720.00"
True Negative Loss Prevention (per Customer),R$8.59,R$8.51


In [8]:
#| label: compare-class
#| echo: true
#| code-fold: true
#| code-summary: "Mostrar/esconder código"
business_results = pd.DataFrame(
    data = [
        rf.classification_metrics,
        hgb.classification_metrics
    ],
    index = [
        "Random Forest",
        "Histogram-based Gradient Boosting"
    ]
)

business_results.transpose().applymap(lambda x: f"{x:,.4f}")

,Random Forest,Histogram-based Gradient Boosting
Classification Threshold,0.6601,0.6974
ROC AUC,0.8397,0.8453
Precision,0.1753,0.1755
Recall,0.5426,0.5891
F1,0.2649,0.2705
Accuracy,0.8809,0.8742


## Variáveis mais importantes do modelo campeão

In [9]:
#| label: feature-positive
#| echo: true
#| code-fold: true
#| code-summary: "Mostrar/esconder código"
feature_importances = hgb \
    .get_feature_importances(test) \
    .reset_index(drop=True)

feature_importances[feature_importances['Importance'] > 0]

,Feature,Importance
0,var15,31859.333333
1,saldo_var30,14622.333333
2,num_var45_hace3,2507.333333
3,saldo_var5,2369.666667
4,var38,2126.000000
5,sum_of_num,1369.666667
6,saldo_medio_var5_ult3,1255.666667
7,sum_of_saldo,1182.333333
8,saldo_medio_var5_hace2,1092.333333
9,num_meses_var5_ult3,983.666667


In [10]:
#| label: feature-custom
#| echo: true
#| code-fold: true
#| code-summary: "Mostrar/esconder código"
feature_importances \
    .where(
        lambda ldf:
        ldf["Feature"].apply(
            lambda row:
            (
                row.startswith("none_")
                | row.startswith("sum_")
                | row.startswith("non_")
            )
        )
    ) \
    .dropna()

,Feature,Importance
5,sum_of_num,1369.666667
7,sum_of_saldo,1182.333333
18,sum_of_imp,142.000000
23,non_zero_count_num,82.666667
29,non_zero_count_saldo,27.666667
32,non_zero_count_ind,20.666667
120,non_zero_count_imp,0.000000
121,none_count_delta,0.000000
122,non_zero_count_delta,0.000000
297,sum_of_delta,-88.666667


## Rankeamento da base de testes

In [11]:
#| label: ranking
#| echo: true
#| code-fold: true
#| code-summary: "Mostrar/esconder código"
test \
    .copy() \
    .assign(predicted=hgb.predict(test.drop("TARGET",axis=1))) \
    .assign(probability=hgb.predict_proba(test.drop("TARGET",axis=1))) \
    .assign(
        profit=(
            lambda ldf:
            ((ldf["TARGET"] * 100) - 10) * ldf["predicted"]
        )
    ) \
    .assign(rank=hgb.rank_customers(test)) \
    [["rank","profit","probability"]] \
    .groupby("rank") \
    .agg({
        "profit": ["mean","sum","count"],
        "probability": ["min"]
    }) \
    .droplevel(0,axis=1) \
    .rename(
        mapper={
            "mean": "avg_profit",
            "sum": "total_profit",
            "count": "total_customers",
            "min": "min_probability"
        },
        axis=1
    )

,avg_profit,total_profit,total_customers,min_probability
rank,,,,
1,7.551506,19060,2524,0.697425
2,0.000000,0,1843,0.523037
3,0.000000,0,3188,0.348692
4,0.000000,0,5474,0.174363
5,0.000000,0,5976,0.027378
